In [19]:
import requests
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import re
import numpy as np

In [20]:
url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/critics:certified_fresh~sort:popular'
r = requests.get(url)

soup = BeautifulSoup(r.text,'html5lib')

table = soup.find('div',attrs = {'class':"discovery-tiles", 'data-id':"movies_in_theaters_critics:certified_fresh~sort:popular"})
data = {'Name':[],'Link':[]}

for row in table.find_all_next('div',attrs = {'class':'js-tile-link'}):
    data['Name'].append(row.img['alt'])
    data['Link'].append(row.a['href'])

df = pd.DataFrame(data)

In [21]:
df

,Name,Link
0,Blue Beetle,/m/blue_beetle
1,Barbie,/m/barbie
2,Oppenheimer,/m/oppenheimer_2023
3,Talk to Me,/m/talk_to_me_2023
4,Teenage Mutant Ninja Turtles: Mutant Mayhem,/m/teenage_mutant_ninja_turtles_mutant_mayhem
5,Elemental,/m/elemental_2023
6,Asteroid City,/m/asteroid_city
7,Spider-Man: Across the Spider-Verse,/m/spider_man_across_the_spider_verse
8,"Mission: Impossible - Dead Reckoning, Part One",/m/mission_impossible_dead_reckoning_part_one
9,Oldboy,/m/oldboy


In [32]:
m = {'Name':[],'Audience Score':[],'Critics Score':[],'Genre':[],'Cast':[]}

def get_details(name,link):    
    m_url = 'https://www.rottentomatoes.com'+link
    m_r = requests.get(m_url)
    soup = BeautifulSoup(m_r.text,'html5lib')

    rating = soup.find('score-board', attrs = {'data-qa':'score-panel'})
    a_score = rating['audiencescore']
    if a_score == '':
        m['Audience Score'].append(np.nan)
    else:
        m['Audience Score'].append(rating['audiencescore'])
    c_score = rating['tomatometerscore']
    if c_score == '':
        m['Critics Score'].append(np.nan)
    else:
        m['Critics Score'].append(rating['tomatometerscore'])

    gen = soup.find('p',attrs = {'data-qa':'score-panel-subtitle'})
    g = gen.text.split(',')[1]
    m['Genre'].append(g.split('/'))

    crew = soup.find('div',attrs = {'class':'cast-wrap'})
    c = []
    for row in crew.find_all_next('a',attrs = {'data-qa':'cast-crew-item-link'}):
        c.append(row.text.strip())
    
    m['Cast'].append(c)
    m['Name'].append(name)

for i in range(len(df)):
    get_details(df['Name'][i],df['Link'][i])

movies = pd.DataFrame(m)

In [33]:
movies

,Name,Audience Score,Critics Score,Genre,Cast
0,Blue Beetle,92,76,"[ Action, Adventure]","[Xolo Maridueña, Adriana Barraza, Damián Alcáz..."
1,Barbie,83,88,[ Comedy],"[Margot Robbie, Ryan Gosling, America Ferrera,..."
2,Oppenheimer,91,93,"[ History, Drama]","[Cillian Murphy, Emily Blunt, Robert Downey Jr..."
3,Talk to Me,82,95,"[ Horror, Mystery & thriller]","[Sophie Wilde, Miranda Otto, Otis Dhanji, Alex..."
4,Teenage Mutant Ninja Turtles: Mutant Mayhem,91,97,"[ Kids & family, Comedy]","[Micah Abbey, Shamon Brown Jr., Nicolas Cantu,..."
5,Elemental,93,74,"[ Kids & family, Comedy]","[Leah Lewis, Mamoudou Athie, Ronnie del Carmen..."
6,Asteroid City,62,75,"[ Comedy, Drama]","[Jason Schwartzman, Scarlett Johansson, Tom Ha..."
7,Spider-Man: Across the Spider-Verse,94,95,"[ Kids & family, Action]","[Shameik Moore, Hailee Steinfeld, Oscar Isaac,..."
8,"Mission: Impossible - Dead Reckoning, Part One",94,96,"[ Action, Adventure]","[Tom Cruise, Hayley Atwell, Ving Rhames, Simon..."
9,Oldboy,94,83,[ Mystery & thriller],"[Choi Min-sik, Yoo Ji-tae, Kang Hye-jung, Ji D..."


In [30]:
m_url = 'https://www.rottentomatoes.com'+'/m/blue_beetle'
m_r = requests.get(m_url)
soup = BeautifulSoup(m_r.text,'html5lib')
cast = []
crew = soup.find('div',attrs = {'class':'cast-wrap'})

for row in crew.find_all_next('a',attrs = {'data-qa':'cast-crew-item-link'}):
    cast.append(row.text.strip())

print(cast)

['Xolo Maridueña', 'Adriana Barraza', 'Damián Alcázar', 'Elpidia Carrillo', 'Becky G', 'Bruna Marquezine', 'Raoul Max Trujillo', 'Susan Sarandon', 'George Lopez', 'Belissa Escobedo', 'Harvey Guillen', 'Angel Manuel Soto', 'Gareth Dunnet-Alcocer', 'John Rickard', 'Zev Foreman', 'Walter Hamada', 'Galen Vaisman', 'Garrett Grant', 'Pawel Pogorzelski', 'Craig Alpert', 'Bobby Krlic', 'Jon Billington', 'Nathan Krochmal', 'Mayes C. Rubeo']


In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/" xmlns="http://www.w3.org/1999/xhtml">
 <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
  <script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-WEHwEli88wqOiQd913F1utFZiwisa8XhCkbjLnbKEpFa/WbFcPKeGg7h4fdsv0Z/" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() { }
  </script>
  <script ccpa-opt-out-geo="US" ccpa-opt-out-ids="dummy" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
  </script>
  <script src="/assets/pizza-pie/javascripts/bundles/roma/rt-common.js?single">
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Con